In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from dotenv import load_dotenv
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import pandas as pd

load_dotenv("./.env")

client_id = os.environ.get("SPOTIFY_CLIENT_ID")
client_secret = os.environ.get("SPOTIFY_CLIENT_SECRET")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id, client_secret), requests_timeout=45)

In [8]:
df = pd.read_csv('features_clustered.csv')
df = df.drop(columns='cluster')
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.671,0.1530,0,-13.569,1,0.0404,0.912000,0.000055,0.0771,0.6580,85.600,audio_features,3oQomOPRNQ5NVFUmLJHbAV,spotify:track:3oQomOPRNQ5NVFUmLJHbAV,https://api.spotify.com/v1/tracks/3oQomOPRNQ5N...,https://api.spotify.com/v1/audio-analysis/3oQo...,211933,4
1,0.122,0.0261,4,-32.465,0,0.0339,0.825000,0.903000,0.1090,0.0323,95.484,audio_features,464ZqN4UqyJTbibnQfsBeG,spotify:track:464ZqN4UqyJTbibnQfsBeG,https://api.spotify.com/v1/tracks/464ZqN4UqyJT...,https://api.spotify.com/v1/audio-analysis/464Z...,424089,4
2,0.432,0.5550,2,-7.336,1,0.0330,0.625000,0.000038,0.1280,0.5270,79.369,audio_features,7FhBSLKkjTFTcqsnDFye9d,spotify:track:7FhBSLKkjTFTcqsnDFye9d,https://api.spotify.com/v1/tracks/7FhBSLKkjTFT...,https://api.spotify.com/v1/audio-analysis/7FhB...,222307,4
3,0.621,0.8260,2,-5.786,1,0.0334,0.002750,0.058300,0.2280,0.7170,119.989,audio_features,5HvN5wQFNTNxlNBgeaPGbf,spotify:track:5HvN5wQFNTNxlNBgeaPGbf,https://api.spotify.com/v1/tracks/5HvN5wQFNTNx...,https://api.spotify.com/v1/audio-analysis/5HvN...,204693,4
4,0.569,0.4450,0,-5.159,1,0.0283,0.307000,0.000003,0.0957,0.4780,78.993,audio_features,4w9jtaewB2iRNCeEKnaAP5,spotify:track:4w9jtaewB2iRNCeEKnaAP5,https://api.spotify.com/v1/tracks/4w9jtaewB2iR...,https://api.spotify.com/v1/audio-analysis/4w9j...,311771,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76229,0.616,0.9650,2,-3.977,1,0.2360,0.026800,0.729000,0.7220,0.2210,150.013,audio_features,1mcBAXFx7CsmRX7RzVtO4d,spotify:track:1mcBAXFx7CsmRX7RzVtO4d,https://api.spotify.com/v1/tracks/1mcBAXFx7Csm...,https://api.spotify.com/v1/audio-analysis/1mcB...,367105,4
76230,0.594,0.6810,0,-10.478,1,0.0399,0.000542,0.577000,0.3620,0.3530,135.015,audio_features,3m7UjLW8iBQcv981SOrstz,spotify:track:3m7UjLW8iBQcv981SOrstz,https://api.spotify.com/v1/tracks/3m7UjLW8iBQc...,https://api.spotify.com/v1/audio-analysis/3m7U...,592960,4
76231,0.559,0.5110,11,-10.862,0,0.0485,0.693000,0.000000,0.3290,0.6570,131.426,audio_features,4vxR30GtRws84IKEvZxOZd,spotify:track:4vxR30GtRws84IKEvZxOZd,https://api.spotify.com/v1/tracks/4vxR30GtRws8...,https://api.spotify.com/v1/audio-analysis/4vxR...,306000,3
76232,0.625,0.9390,2,-5.278,1,0.0334,0.740000,0.101000,0.1880,0.7590,102.948,audio_features,0q5ARDwy987big1XTriY21,spotify:track:0q5ARDwy987big1XTriY21,https://api.spotify.com/v1/tracks/0q5ARDwy987b...,https://api.spotify.com/v1/audio-analysis/0q5A...,188920,4


In [99]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import numpy as np

def getClusterIds(df):
    # Normalize the data
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(df[["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]])
    

    # Perform PCA
    # pca = PCA(n_components=9)
    # pca.fit(X_normalized)
    # X_pca = pca.transform(X_normalized)

    # Predict clusters using KMeans
    kmeans = KMeans(n_clusters=1000, n_init=10) # TODO: Find optimal number of clusters using an elbow graph
    kmeans.fit(X_normalized)
    cluster_ids = kmeans.predict(X_normalized)
    return cluster_ids, X_normalized, kmeans

cluster_ids, X_normalized, kmeans = getClusterIds(df)

# Plot clusters
#for c in np.unique(cluster_ids):
    #plt.scatter(X_pca[cluster_ids == c, 0], X_pca[cluster_ids == c, 1], marker='.', s=10)

In [100]:
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np


In [101]:
import pickle

with open("kmeans1000.pickle", "wb") as f: #saves the model in dir and file name given
    pickle.dump(kmeans,f)

In [92]:
with open("kmeans1000.pickle", "rb") as f: #don't forget the correct path
     pickle.load(f) 

In [85]:
df['cluster'] = cluster_ids
df.cluster.value_counts()

cluster
91      142
1133    133
323     123
126     122
237     118
       ... 
1750      1
878       1
1659      1
1595      1
1450      1
Name: count, Length: 2000, dtype: int64

In [44]:
df['cluster_distance'] = kmeans.transform(X_pca).min(axis=1)

In [58]:
df[df.id.isin(['4cvzNj4FmjNtkY0tSTwufs', '2aUDpZRSit5Zg2R6L5r6dW', '3tXcaUTyYJm5nBpLYp7Tmb'])]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,cluster,cluster_distance
16331,0.334,0.423,1,-13.562,1,0.0348,0.0173,0.994,0.0732,0.0859,100.058,audio_features,2aUDpZRSit5Zg2R6L5r6dW,spotify:track:2aUDpZRSit5Zg2R6L5r6dW,https://api.spotify.com/v1/tracks/2aUDpZRSit5Z...,https://api.spotify.com/v1/audio-analysis/2aUD...,93227,4,270,2.685626
49778,0.764,0.975,0,-5.680,1,0.1760,0.0520,0.512,0.2540,0.1110,150.017,audio_features,3tXcaUTyYJm5nBpLYp7Tmb,spotify:track:3tXcaUTyYJm5nBpLYp7Tmb,https://api.spotify.com/v1/tracks/3tXcaUTyYJm5...,https://api.spotify.com/v1/audio-analysis/3tXc...,363200,4,450,2.630694
74477,0.327,0.345,6,-13.282,0,0.0310,0.0125,0.833,0.1500,0.0364,200.096,audio_features,4cvzNj4FmjNtkY0tSTwufs,spotify:track:4cvzNj4FmjNtkY0tSTwufs,https://api.spotify.com/v1/tracks/4cvzNj4FmjNt...,https://api.spotify.com/v1/audio-analysis/4cvz...,175256,4,456,3.518514


In [69]:
df.to_csv('features_clustered2.csv')

In [84]:
df[df['id']== '2dHHgzDwk4BJdRwy9uXhTO']

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,cluster,cluster_distance


In [74]:
K = range(5, 40) #range of the clusters
inertia = []

 # Normalize the data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(df[["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]])
 # df_normalized = pd.DataFrame(X_normalized, columns=["danceability", "energy",  "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"])



for k in K: #for every data in set build a inertia
    print("Training a K-Means model with {} clusters! ".format(k))
    print()
    kmeans1 = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans1.fit(X_normalized)
    inertia.append(kmeans1.inertia_)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0)) #changing the scale
plt.title('Elbow Method showing the optimal k') 

ValueError: could not convert string to float: '3oQomOPRNQ5NVFUmLJHbAV'